In [1]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import cartopy.crs as ccrs
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import matplotlib.patches as patches
import cmaps
import cartopy.feature as cfeature
from datetime import datetime,timedelta
from matplotlib.offsetbox import AnchoredText#关于修改text位置的
import cv2
from pygac import get_reader_class
import ast
from utiles import *
from tqdm import tqdm
proj =ccrs.NorthPolarStereo(central_longitude=17.5)
from sklearn.neighbors import BallTree
from shapely.geometry import Point, Polygon,LineString
#经过实验，按经纬度插值的清晰度小于先转化为实际地理坐标系再插值的清晰度
import concurrent.futures
import pyproj
import matplotlib.style as mplstyle
from matplotlib.colors import BoundaryNorm
from datetime import datetime

In [2]:
def withdraw_info(filename):
    #根据ASCAT文件名提取信息
    info_list = filename.split('_')
    year = int(info_list[1][0:4])
    month = int(info_list[1][4:6])
    day = int(info_list[1][6:8])
    start_hour = int(info_list[2][0:2])
    start_minute = int(info_list[2][2:4])
    start_time = pd.Timestamp(year=year,month=month,day=day,hour=start_hour,minute=start_minute)
    satellite_name = info_list[3]
    return start_time,satellite_name

In [3]:
def geo2lonlat(central_lon,central_lat,distance_x,distance_y):
    #位移不变的经纬度转换
    earth_radius = 6370856#m
    lats = distance_y/(2*np.pi*earth_radius)*360+central_lat
    lon_diff = distance_x/(2*np.pi*earth_radius*np.cos(central_lat/180*np.pi))*360
    return central_lon+lon_diff,lats

In [4]:
#风速向北吹为0°,顺时针旋转
def angel2uv(wind_dir,wind_speed):
    rad = np.radians(wind_dir)
    u = wind_speed * np.sin(rad)
    v = wind_speed * np.cos(rad)
    return u,v

In [6]:
def calculate_meantime(filepath,tle_name,central_lon,central_lat):
    #计算中心点50KM内的平均扫描时间
    tle_dir = r'Z:\NOAA_TLE'
    reader_cls = get_reader_class(filepath)
    reader = reader_cls(tle_dir=tle_dir, tle_name=tle_name)
    reader.read(filepath)
    times = reader.get_times()
    lons, lats = reader.get_lonlat()
    dis_array = geodistance(central_lon,central_lat,lons,lats)
    scantime150 = times[(dis_array<50).sum(axis=1)>0]
    #去除空值
    scantime150 = scantime150[~np.isnan(scantime150)]
    if len(scantime150)==0:
        return np.nan
    meantime = pd.Series(scantime150).mean()
    #规整到分钟
    return pd.to_datetime( meantime.strftime('%Y%m%d_%H%M'),format='%Y%m%d_%H%M')

In [5]:
#这里使用的是我们已经校验好的气旋特征
cloud_infos = pd.read_excel(r"D:\Desktop\pycode\PLs_identity\IMPMCT\example_data\cloud_infos.xlsx")
cloud_infos

,label,x1,y1,x2,y2,x3,y3,x4,y4,x0,...,path_index,filepath,tle_name,dataset_type,satellite_name,mean_time,cloud_lon,cloud_lat,AVHRR_filename,step
0,comma,270.457,477.547,397.375,598.625,581.900,405.199,454.983,284.121,420.500,...,1070,Z:\NOAA_AVHRR\NSS.GHRR.NM.D08321.S1302.E1451.B...,NOAA-17_tle.txt,GHRR,NOAA-17,2008-11-16 13:55:00,18.083467,74.624092,NSS.GHRR.NM.D08321.S1302.E1451.B3324950.WI,1
1,comma,351.750,603.625,564.184,417.948,445.128,281.735,232.694,467.412,401.750,...,1070,Z:\NOAA_AVHRR\NSS.GHRR.NL.D08321.S1239.E1413.B...,NOAA-16_tle.txt,GHRR,NOAA-16,2008-11-16 14:09:00,18.051287,74.612850,NSS.GHRR.NL.D08321.S1239.E1413.B4202829.GC,2
2,comma,383.000,580.500,556.973,394.689,435.781,281.218,261.808,467.029,403.000,...,1070,Z:\NOAA_AVHRR\NSS.GHRR.NK.D08321.S1324.E1508.B...,NOAA-15_tle.txt,GHRR,NOAA-15,2008-11-16 14:15:00,18.136766,74.680301,NSS.GHRR.NK.D08321.S1324.E1508.B5465051.GC,3
3,comma,222.463,483.334,342.297,622.231,582.500,414.996,462.666,276.099,421.343,...,1070,Z:\NOAA_AVHRR\NSS.GHRR.NM.D08321.S1447.E1632.B...,NOAA-17_tle.txt,GHRR,NOAA-17,2008-11-16 16:24:00,18.869213,74.365530,NSS.GHRR.NM.D08321.S1447.E1632.B3325051.WI,4
4,comma,243.140,490.784,381.125,621.125,587.807,402.321,449.821,271.980,424.875,...,1070,Z:\NOAA_AVHRR\NSS.GHRR.M2.D08321.S1501.E1641.B...,Metop-A_tle.txt,GHRR,Metop-A,2008-11-16 16:39:00,19.106939,74.441543,NSS.GHRR.M2.D08321.S1501.E1641.B1078283.SV,5
5,comma,247.795,485.612,381.196,609.250,567.418,408.323,434.016,284.685,418.071,...,1070,Z:\NOAA_AVHRR\NSS.GHRR.NM.D08321.S1627.E1807.B...,NOAA-17_tle.txt,GHRR,NOAA-17,2008-11-16 18:03:00,19.648987,74.430301,NSS.GHRR.NM.D08321.S1627.E1807.B3325152.GC,6
6,comma,261.468,468.107,394.875,591.750,580.411,391.563,447.004,267.920,423.696,...,1070,Z:\NOAA_AVHRR\NSS.GHRR.M2.D08321.S1640.E1822.B...,Metop-A_tle.txt,GHRR,Metop-A,2008-11-16 18:19:00,20.027585,74.475268,NSS.GHRR.M2.D08321.S1640.E1822.B1078384.SV,7
7,comma,271.541,477.210,418.000,593.000,588.174,377.754,441.715,261.964,431.018,...,1070,Z:\NOAA_AVHRR\NSS.GHRR.NM.D08321.S1802.E1947.B...,NOAA-17_tle.txt,GHRR,NOAA-17,2008-11-16 18:54:00,20.520402,74.497752,NSS.GHRR.NM.D08321.S1802.E1947.B3325253.GC,8
8,comma,256.644,502.678,392.136,609.798,557.518,400.614,422.025,293.494,416.786,...,1070,Z:\NOAA_AVHRR\NSS.GHRR.M2.D08321.S1821.E2003.B...,Metop-A_tle.txt,GHRR,Metop-A,2008-11-16 20:00:00,20.562498,74.452785,NSS.GHRR.M2.D08321.S1821.E2003.B1078485.SV,9
9,comma,282.429,555.545,424.250,604.250,517.644,332.302,375.823,283.597,403.625,...,1070,Z:\NOAA_AVHRR\NSS.GHRR.NN.D08322.S0132.E0326.B...,NOAA-18_tle.txt,GHRR,NOAA-18,2008-11-17 01:53:00,23.929505,74.770235,NSS.GHRR.NN.D08322.S0132.E0326.B1800304.SV,10


In [7]:
#气旋中心精确时间计算
cloud_times = []
for index,cloud_info in tqdm(cloud_infos.iterrows()):
    cloud_times.append(calculate_meantime(cloud_info.filepath,cloud_info.tle_name,cloud_info.cloud_lon,cloud_info.cloud_lat))
cloud_infos['cloud_time'] = cloud_times 

33it [00:27,  1.20it/s]


气旋大小特征

In [8]:
#根据坐标计算点到直线距离
def get_distance_from_point_to_line(x0,y0, line):
    A = line[:,3]-line[:,1]#A=Y2-Y1
    B = line[:,0]-line[:,2]#B=X1-X2
    C = -(A * line[:,0] + B * line[:,1])#C=X1*(Y1-Y2)+Y1*(X2-X1)
    #根据点到直线的距离公式计算距离
    distance = np.abs(A * x0 + B * y0 + C) / (np.sqrt(A**2 + B**2))
    return distance

In [9]:
#长度和宽度,x1-x4,y1,y4逆时针旋转
w = np.sqrt((cloud_infos.x1-cloud_infos.x2)**2+(cloud_infos.y1-cloud_infos.y2)**2).values
h= np.sqrt((cloud_infos.x1-cloud_infos.x4)**2+(cloud_infos.y1-cloud_infos.y4)**2).values
r = np.concatenate([h.reshape((-1,1)),w.reshape((-1,1))],axis=1)
#width和length分别为矩形短边和长边
cloud_infos['width'],cloud_infos['length'] = np.min(r,axis=1)*2,np.max(r,axis=1)*2#KM

In [10]:
#计算云中心至短边的长距离和短距离
height_short_bool,width_short_bool= cloud_infos['width']==h*2,cloud_infos['width']==w*2
short_line1 = cloud_infos.loc[height_short_bool,['x2','y2','x3','y3']].values
short_line2 = cloud_infos.loc[height_short_bool,['x1','y1','x4','y4']].values
short_line3 = cloud_infos.loc[width_short_bool,['x1','y1','x2','y2']].values
short_line4 = cloud_infos.loc[width_short_bool,['x4','y4','x3','y3']].values
dis1 = get_distance_from_point_to_line(cloud_infos.loc[height_short_bool,'x0'].values,cloud_infos.loc[height_short_bool,'y0'].values, short_line1)
dis2 = get_distance_from_point_to_line(cloud_infos.loc[height_short_bool,'x0'].values,cloud_infos.loc[height_short_bool,'y0'].values, short_line2)
dis3 = get_distance_from_point_to_line(cloud_infos.loc[width_short_bool,'x0'].values,cloud_infos.loc[width_short_bool,'y0'].values, short_line3)
dis4 = get_distance_from_point_to_line(cloud_infos.loc[width_short_bool,'x0'].values,cloud_infos.loc[width_short_bool,'y0'].values, short_line4)
cloud_infos.loc[height_short_bool,'dis1'] = dis1
cloud_infos.loc[height_short_bool,'dis2'] = dis2
cloud_infos.loc[width_short_bool,'dis1'] = dis3
cloud_infos.loc[width_short_bool,'dis2'] = dis4
cloud_infos['l_search'] = np.max(cloud_infos[['dis1','dis2']],axis=1)*2
cloud_infos['s_search'] = np.min(cloud_infos[['dis1','dis2']],axis=1)*2
cloud_infos

,label,x1,y1,x2,y2,x3,y3,x4,y4,x0,...,cloud_lat,AVHRR_filename,step,cloud_time,width,length,dis1,dis2,l_search,s_search
0,comma,270.457,477.547,397.375,598.625,581.900,405.199,454.983,284.121,420.500,...,74.624092,NSS.GHRR.NM.D08321.S1302.E1451.B3324950.WI,1,2008-11-16 13:55:00,350.816538,534.653017,153.890536,113.435734,307.781072,226.871469
1,comma,351.750,603.625,564.184,417.948,445.128,281.735,232.694,467.412,401.750,...,74.612850,NSS.GHRR.NL.D08321.S1239.E1413.B4202829.GC,2,2008-11-16 14:09:00,361.819361,564.284158,116.166605,165.975474,331.950948,232.333210
2,comma,383.000,580.500,556.973,394.689,435.781,281.218,261.808,467.029,403.000,...,74.680301,NSS.GHRR.NK.D08321.S1324.E1508.B5465051.GC,3,2008-11-16 14:14:00,332.043182,509.086761,112.671682,141.871698,283.743396,225.343365
3,comma,222.463,483.334,342.297,622.231,582.500,414.996,462.666,276.099,421.343,...,74.365530,NSS.GHRR.NM.D08321.S1447.E1632.B3325051.WI,4,2008-11-16 16:24:00,366.892705,634.488223,199.482674,117.761438,398.965348,235.522875
4,comma,243.140,490.784,381.125,621.125,587.807,402.321,449.821,271.980,424.875,...,74.441543,NSS.GHRR.M2.D08321.S1501.E1641.B1078283.SV,5,2008-11-16 16:39:00,379.624217,601.970850,187.700963,113.284725,375.401927,226.569449
5,comma,247.795,485.612,381.196,609.250,567.418,408.323,434.016,284.685,418.071,...,74.430301,NSS.GHRR.NM.D08321.S1627.E1807.B3325152.GC,6,2008-11-16 18:03:00,363.770157,547.904810,174.962019,98.990647,349.924038,197.981295
6,comma,261.468,468.107,394.875,591.750,580.411,391.563,447.004,267.920,423.696,...,74.475268,NSS.GHRR.M2.D08321.S1640.E1822.B1078384.SV,7,2008-11-16 18:19:00,363.785756,545.888055,158.485950,114.458078,316.971899,228.916156
7,comma,271.541,477.210,418.000,593.000,588.174,377.754,441.715,261.964,431.018,...,74.497752,NSS.GHRR.NM.D08321.S1802.E1947.B3325253.GC,8,2008-11-16 18:54:00,373.403604,548.780578,158.590729,115.799560,317.181458,231.599120
8,comma,256.644,502.678,392.136,609.798,557.518,400.614,422.025,293.494,416.786,...,74.452785,NSS.GHRR.M2.D08321.S1821.E2003.B1078485.SV,9,2008-11-16 20:00:00,345.443347,533.324745,177.020634,89.641969,354.041268,179.283939
9,comma,282.429,555.545,424.250,604.250,517.644,332.302,375.823,283.597,403.625,...,74.770235,NSS.GHRR.NN.D08322.S0132.E0326.B1800304.SV,10,2008-11-17 01:53:00,299.902471,575.076183,186.594836,100.943255,373.189672,201.886510


匹配风速

In [11]:
def near_judge(chosed_cloud_info,chosed_wind_urlinfo):
    #匹配ASCAT和quickscat数据
    result = [] 
    chosed_time,chosed_lon,chosed_lat = chosed_cloud_info[['cloud_time','cloud_lon','cloud_lat']]
    chosed_time = pd.Timestamp(chosed_time)
    wind_data = xr.open_dataset(chosed_wind_urlinfo.path)
    #将风速经纬度数据统一转化为-180至180°
    lons,lats,times = wind_data.lon.values,wind_data.lat.values,wind_data.time
    lons = np.where(lons>180,lons-360,lons)
    disses = geodistance(chosed_lon,chosed_lat,lons,lats)
    near_num= np.nansum(disses<50)
    #当小于50KM的风速数据大于20个，且平均扫描时间在当前时间内，保留信息
    if near_num>0:
        if chosed_wind_urlinfo.satellite=='QS':
            scantimes = times.values[(disses<50).sum(axis=1)>0]
            wind_speed =  wind_data.retrieved_wind_speed.values
            mean_time = pd.Series(scantimes).mean()
        else:
            scantimes = times.values[(disses<50)]
            scantimes = scantimes[~np.isnan(scantimes)]
            wind_speed = wind_data.wind_speed.values
            mean_time = pd.Series(scantimes).mean()
        wind_near_num= np.nansum(~np.isnan(wind_speed[disses<50]))
        #当风速数据不够时，不保留信息
        if wind_near_num<10:
            return result
        #当扫描时间在30分钟外，不保留信息
        if mean_time>chosed_time:
            #三十分钟内
            minute_diff = (mean_time-chosed_time).total_seconds()/60
        else:
            minute_diff = (chosed_time-mean_time).total_seconds()/60
        if minute_diff<30:
            chosed_cloud_info['wind_path'] = chosed_wind_urlinfo.path
            chosed_cloud_info['wind_mean_time'] = pd.to_datetime( mean_time.strftime('%Y%m%d_%H%M'),format='%Y%m%d_%H%M')
            chosed_cloud_info['wind_satellite'] =  chosed_wind_urlinfo.satellite
            chosed_cloud_info['wind_match_num']  =   wind_near_num
            chosed_cloud_info['minutes_diff']  =   minute_diff
            #长搜索半径内75分位数风场和短KM内10m最大风场
            chosed_cloud_info['env_advection_wind']  =  np.nanpercentile(wind_speed[(disses<chosed_cloud_info.l_search)],75)
            #短搜索半径内10m风场
            in_r_bool = (disses<chosed_cloud_info.s_search)
            in_r_lons,in_r_lats,in_r_wind = lons[in_r_bool],lats[in_r_bool],wind_speed[in_r_bool]
            chosed_cloud_info['max_wind']  =  np.nanmax(in_r_wind)
            max_wind_index = np.nanargmax(in_r_wind)
            chosed_cloud_info['wind_lon']  =  in_r_lons[max_wind_index]
            chosed_cloud_info['wind_lat']  = in_r_lats[max_wind_index]
            chosed_cloud_info['valid_fraction']  =  1-np.isnan(in_r_wind).sum()/len(in_r_wind)
            result.append(pd.DataFrame(chosed_cloud_info).T)
    return result

风速计信息提取

In [12]:
#ASCAT信息提取
ASCAT_folder = r"Y:\ASCAT_data"
ASCAT_filenames = os.listdir(ASCAT_folder)
ASCAT_info = pd.DataFrame(ASCAT_filenames,columns=['filename'])
infos = np.array([withdraw_info(os.path.basename(i)) for i in ASCAT_filenames])
ASCAT_info['start_time'] = infos[:,0]
ASCAT_info['satellite'] = 'AS'
ASCAT_info['month'] = ASCAT_info['start_time'].apply(lambda x:x.month)
ASCAT_info['integral_time'] = ASCAT_info['start_time'].apply(lambda x:x-timedelta(seconds=x.minute*60)+timedelta(seconds=60*60)*round(x.minute/60))#将分钟去掉
ASCAT_info['path'] = [os.path.join(ASCAT_folder,i) for i in ASCAT_filenames]
#qs信息提取
qs_folder = r"Y:\qs_data"
qs_filenames = os.listdir(qs_folder)
qs_info = pd.DataFrame(qs_filenames,columns=['filename'])
qs_info['start_time'] =  [datetime.strptime(os.path.basename(i).split('_')[-1].split('.')[0], "%Y%m%d%H%M") for i in qs_filenames]
qs_info['satellite'] = 'QS'
qs_info['month'] = qs_info['start_time'].apply(lambda x:x.month)
qs_info['integral_time'] = qs_info['start_time'].apply(lambda x:x-timedelta(seconds=x.minute*60)+timedelta(seconds=60*60)*round(x.minute/60))#将分钟去掉
qs_info['path'] = [os.path.join(qs_folder,i) for i in qs_filenames]
wind_info = pd.concat([qs_info,ASCAT_info],axis=0).reset_index(drop=True)
wind_info

,filename,start_time,satellite,month,integral_time,path
0,qs_l2b_02784_v4.1_199912312316.nc4,1999-12-31 23:16:00,QS,12,1999-12-31 23:00:00,Y:\qs_data\qs_l2b_02784_v4.1_199912312316.nc4
1,qs_l2b_02785_v4.1_200001010057.nc4,2000-01-01 00:57:00,QS,1,2000-01-01 01:00:00,Y:\qs_data\qs_l2b_02785_v4.1_200001010057.nc4
2,qs_l2b_02786_v4.1_200001010238.nc4,2000-01-01 02:38:00,QS,1,2000-01-01 03:00:00,Y:\qs_data\qs_l2b_02786_v4.1_200001010238.nc4
3,qs_l2b_02787_v4.1_200001010419.nc4,2000-01-01 04:19:00,QS,1,2000-01-01 04:00:00,Y:\qs_data\qs_l2b_02787_v4.1_200001010419.nc4
4,qs_l2b_02788_v4.1_200001010600.nc4,2000-01-01 06:00:00,QS,1,2000-01-01 06:00:00,Y:\qs_data\qs_l2b_02788_v4.1_200001010600.nc4
...,...,...,...,...,...,...
89352,ascat_20250101_202400_metopc_31935_eps_o_coa_3...,2025-01-01 20:24:00,AS,1,2025-01-01 20:00:00,Y:\ASCAT_data\ascat_20250101_202400_metopc_319...
89353,ascat_20250101_210600_metopb_63778_eps_o_coa_3...,2025-01-01 21:06:00,AS,1,2025-01-01 21:00:00,Y:\ASCAT_data\ascat_20250101_210600_metopb_637...
89354,ascat_20250101_220600_metopc_31936_eps_o_coa_3...,2025-01-01 22:06:00,AS,1,2025-01-01 22:00:00,Y:\ASCAT_data\ascat_20250101_220600_metopc_319...
89355,ascat_20250101_224800_metopb_63779_eps_o_coa_3...,2025-01-01 22:48:00,AS,1,2025-01-01 23:00:00,Y:\ASCAT_data\ascat_20250101_224800_metopb_637...


In [13]:
#匹配风速
results = []
for i in tqdm(range(len(cloud_infos))):
    chosed_cloud_info = cloud_infos.loc[i].copy()
    chosed_time,chosed_lon,chosed_lat = chosed_cloud_info[['cloud_time','cloud_lon','cloud_lat']]
    chosed_time = pd.Timestamp(chosed_time)
    #这里把时间定为两个小时，因为从文件名只能得到相隔较久的起始时间
    candidate_wind_info = wind_info.loc[np.logical_and(wind_info.start_time<chosed_time,
                                            wind_info.start_time>(chosed_time-timedelta(hours=4)))]
    #对每个可能轨迹进行匹配
    for j in range(len(candidate_wind_info)):
        chosed_wind_urlinfo = candidate_wind_info.iloc[j]
        temp_result = near_judge(chosed_cloud_info,chosed_wind_urlinfo)
        if len(temp_result):
            results+=temp_result
len(results)

100%|██████████| 33/33 [00:08<00:00,  4.09it/s]


12

In [14]:
wind = pd.concat(results,axis=0).infer_objects()
wind

,label,x1,y1,x2,y2,x3,y3,x4,y4,x0,...,wind_path,wind_mean_time,wind_satellite,wind_match_num,minutes_diff,env_advection_wind,max_wind,wind_lon,wind_lat,valid_fraction
3,comma,222.463,483.334,342.297,622.231,582.500,414.996,462.666,276.099,421.343,...,Y:\qs_data\qs_l2b_49016_v4.1_200811161552.nc4,2008-11-16 16:47:00,QS,11,23.710237,9.226493,17.217941,20.363583,75.125702,0.854271
4,comma,243.140,490.784,381.125,621.125,587.807,402.321,449.821,271.980,424.875,...,Y:\qs_data\qs_l2b_49016_v4.1_200811161552.nc4,2008-11-16 16:47:00,QS,17,8.694686,9.492586,17.217941,20.363583,75.125702,0.846154
5,comma,247.795,485.612,381.196,609.250,567.418,408.323,434.016,284.685,418.071,...,Y:\qs_data\qs_l2b_49017_v4.1_200811161733.nc4,2008-11-16 18:28:00,QS,49,25.211527,10.743968,21.192593,18.448545,74.754066,1.000000
6,comma,261.468,468.107,394.875,591.750,580.411,391.563,447.004,267.920,423.696,...,Y:\qs_data\qs_l2b_49017_v4.1_200811161733.nc4,2008-11-16 18:28:00,QS,52,9.180427,11.076945,21.192593,18.448545,74.754066,0.981238
7,comma,271.541,477.210,418.000,593.000,588.174,377.754,441.715,261.964,431.018,...,Y:\qs_data\qs_l2b_49017_v4.1_200811161733.nc4,2008-11-16 18:28:00,QS,50,25.850674,11.048266,21.192593,18.448545,74.754066,0.970000
8,comma,256.644,502.678,392.136,609.798,557.518,400.614,422.025,293.494,416.786,...,Y:\qs_data\qs_l2b_49018_v4.1_200811161914.nc4,2008-11-16 20:07:00,QS,49,7.950466,10.370130,17.779675,19.933359,74.798431,1.000000
11,comma,294.451,569.917,439.250,613.000,521.538,336.433,376.739,293.350,403.000,...,Y:\qs_data\qs_l2b_49023_v4.1_200811170339.nc4,2008-11-17 04:25:00,QS,30,2.782228,12.541996,16.811314,23.126719,74.885506,0.847603
12,comma,291.746,550.813,439.875,601.750,532.170,333.350,384.041,282.413,411.125,...,Y:\qs_data\qs_l2b_49023_v4.1_200811170339.nc4,2008-11-17 04:25:00,QS,18,16.155571,12.587092,16.811314,23.126719,74.885506,0.816733
29,comma,360.629,653.479,550.500,649.250,541.951,265.447,352.080,269.676,397.375,...,Y:\qs_data\qs_l2b_49030_v4.1_200811171526.nc4,2008-11-17 16:21:00,QS,15,21.129301,13.472493,22.877974,31.503637,74.806839,0.625000
30,comma,360.045,654.034,546.750,649.875,538.256,268.555,351.551,272.713,396.125,...,Y:\qs_data\qs_l2b_49030_v4.1_200811171526.nc4,2008-11-17 16:21:00,QS,10,3.113750,13.319740,22.877974,31.503637,74.806839,0.600000
